In [3]:
import requests
import pandas as pd
import json


In [4]:
FOURSQUARE_API_KEY = 'your_foursquare_api_key'
foursquare_headers = {
    "Accept": "application/json",
    "Authorization": FOURSQUARE_API_KEY
}
foursquare_base_url = "https://api.foursquare.com/v3/places/search"


In [5]:
YELP_API_KEY = 'your_yelp_api_key'
yelp_headers = {
    "Authorization": f"Bearer {YELP_API_KEY}"
}
yelp_base_url = "https://api.yelp.com/v3/businesses/search"


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [9]:
import requests

FOURSQUARE_API_KEY = "fsq3cm08W36JgWuf2QlA0zNC2a+evXleGtvcK954ZJhKTOw="

import pandas as pd

data = {
    'station': ['Station A', 'Station B'],
    'latitude': [40.748817, 37.774929],
    'longitude': [-73.985428, -122.419416]
}

df = pd.DataFrame(data)

latitude = df.iloc[0]['latitude']
longitude = df.iloc[0]['longitude']
url = "https://api.foursquare.com/v3/places/search"
headers = {
    "Accept": "application/json",
    "Authorization": FOURSQUARE_API_KEY
}
params = {
    "ll": f"{latitude},{longitude}",  
    "radius": 1000,                   
    "limit": 10                       
}
response = requests.get(url, headers=headers, params=params)
if response.status_code == 200:
    results = response.json()
    print("✅ Places found near bike station:")
    for place in results['results']:
        print(f"- {place['name']} ({place['location']['address'] if 'address' in place['location'] else 'No address'})")
else:
    print(f"❌ Request failed with status code {response.status_code}")

✅ Places found near bike station:
- Empire State Building (350 5th Ave)
- Yoon Haeundae Galbi (8 W 36th St)
- Grace Street Cafe (17 W 32nd St)
- Tonchin New York (13 W 36th St)
- BCD Tofu House (5 W 32nd St)
- The Parlour Room (70 W 36th St)
- Tina's Cuban Cuisine (179 Madison Ave)
- Ayza Wine & Chocolate Bar (11 W 31st St)
- Don's Bogam BBQ & Wine Bar (17 E 32nd St)
- Ravagh Persian Grill (173 Madison Ave)


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [10]:
foursquare_pois = []

for poi in data.get('results', []):
    foursquare_pois.append({
        "name": poi.get("name"),
        "category": poi['categories'][0]['name'] if poi.get("categories") else None,
        "latitude": poi['geocodes']['main']['latitude'],
        "longitude": poi['geocodes']['main']['longitude'],
        "distance_m": poi.get("distance")
    })


Put your parsed results into a DataFrame

In [11]:
fsq_df = pd.DataFrame(foursquare_pois)
fsq_df.head()


""


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [14]:
import pandas as pd

bike_stations = [
    {"station": "SF Station 1", "latitude": 37.7749, "longitude": -122.4194},
    {"station": "SF Station 2", "latitude": 37.7816, "longitude": -122.4112},
    {"station": "SF Station 3", "latitude": 37.7680, "longitude": -122.4290},
]

bike_df = pd.DataFrame(bike_stations)


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [15]:
yelp_pois = []

for business in data.get("businesses", []):
    yelp_pois.append({
        "station": station_name,  # From the loop earlier
        "name": business.get("name"),
        "category": business["categories"][0]["title"] if business.get("categories") else None,
        "latitude": business["coordinates"]["latitude"],
        "longitude": business["coordinates"]["longitude"],
        "distance_m": business.get("distance"),
        "rating": business.get("rating"),
        "review_count": business.get("review_count")
    })


Put your parsed results into a DataFrame

yelp_df = pd.DataFrame(yelp_results)
yelp_df.head()


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

print("Number of POIs from Yelp:", len(yelp_df))
print("Number of POIs from Foursquare:", len(foursquare_df))


Get the top 10 restaurants according to their rating

In [19]:
import requests
import pandas as pd
import time

YELP_API_KEY ="M2nNMPI4L-7WEcuB0uVLlNLLW0qrwURgQC8Jc51xjhPs3fHCvH0C3f5IMTzTwnHqHZu88A1HQKjLzRrEZ3u4ocWeA7rRQj8YwCA1CNzo0i5MoOcev_LsBrt9_0D7Z3Yx"

headers = {
    "Authorization": f"Bearer {YELP_API_KEY}"
}
yelp_url = "https://api.yelp.com/v3/businesses/search"


In [20]:
bike_stations = [
    {"station": "SF Station 1", "latitude": 37.7749, "longitude": -122.4194},
    {"station": "SF Station 2", "latitude": 37.7816, "longitude": -122.4112},
    {"station": "SF Station 3", "latitude": 37.7680, "longitude": -122.4290},
]

bike_df = pd.DataFrame(bike_stations)


In [21]:
yelp_results = []

for _, row in bike_df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    station_name = row['station']

    params = {
        "latitude": lat,
        "longitude": lon,
        "radius": 1000,
        "categories": "restaurants",
        "limit": 50
    }

    response = requests.get(yelp_url, headers=headers, params=params)
    data = response.json()

    for business in data.get("businesses", []):
        yelp_results.append({
            "station": station_name,
            "name": business.get("name"),
            "category": business["categories"][0]["title"] if business.get("categories") else None,
            "latitude": business["coordinates"]["latitude"],
            "longitude": business["coordinates"]["longitude"],
            "distance_m": business.get("distance"),
            "rating": business.get("rating"),
            "review_count": business.get("review_count")
        })

    time.sleep(1)  


In [22]:
yelp_df = pd.DataFrame(yelp_results)


Get the top 10 restaurants according to their rating

In [23]:
top_10_restaurants = yelp_df.sort_values(by=['rating', 'review_count'], ascending=[False, False]).head(10)
top_10_restaurants[['name', 'category', 'rating', 'review_count', 'distance_m']]


,name,category,rating,review_count,distance_m
35,San Francisco Magic Theater At The Marrakech,Comedy Clubs,5.0,1560,1382.450168
64,San Francisco Magic Theater At The Marrakech,Comedy Clubs,5.0,1560,470.703932
123,San Francisco Magic Theater At The Marrakech,Comedy Clubs,5.0,1560,2488.624347
48,San Francisco Wine & Cheese,Cheese Shops,4.9,214,300.891176
11,Ashyan's Lu Ruo Fan,Taiwanese,4.8,33,3.551888
145,Freekeh,Mediterranean,4.7,250,645.066694
28,Taniku Izakaya,Izakaya,4.7,243,1198.073133
72,Taniku Izakaya,Izakaya,4.7,243,929.423953
82,The Bite,Mediterranean,4.7,219,222.531708
18,Yakiniku Shodai,Japanese,4.7,102,179.182658
